# MANU 465 Capstone Project

The following is the code for MANU 465 EEG Group #9 Capstone Project. The code is written in Python and uses the following libraries:

* Pandas
* Numpy
* Matplotlib
* Seaborn
* Scikit-learn


## Objective

Our objective is to use machine learning to determine if a person is left or right handed based on brainwave data.

## Setup

### Import Libraries

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import tensorflow.keras as kr
import seaborn as sns
import os

### Authors

In [18]:
d = {'Josiah': 7412148, 'Muyang Li': 000000, 'Sofi': 0, 'Marti': 0, 'Lisa': 0, 'Simon': 0}    # please add your name and student ID
Authors = pd.DataFrame(d.items(), columns=['Name', 'Student ID'])
Authors

,Name,Student ID
0,Josiah,7412148
1,Muyang Li,0
2,Sofi,0
3,Marti,0
4,Lisa,0
5,Simon,0


## Importing Raw Data

In [3]:
# Get Current Working directory and append the data relative dir
cwd = os.getcwd()
LHD_dir = cwd + r"Data\Unprocessed\LeftHandedDominant"
RHD_dir = cwd + r"Data\Unprocessed\RightHandDominant"

# Hold file locations
leftHandDominant    = []
rightHandDominant   = []

#Populate file location arrays
for file in os.listdir(LHD_dir):
    if file.endswith('.csv'):
        leftHandDominant.append(os.path.join(LHD_dir, file))
for file in os.listdir(RHD_dir):
        if file.endswith('.csv'):
            rightHandDominant.append(os.path.join(RHD_dir, file))
            
#Test reading files by changing num
num=6
sample = pd.read_csv(leftHandDominant[num])
sample 

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'd:\\Documents\\UBC\\MANU 465\\MANU-465-EEG4Data\\Unprocessed\\LeftHandedDominant'